In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

In [8]:
solar_data = pd.read_csv("C:\\Users\\PRANAV\\Downloads\\Plant_1_Weather_Sensor_Data.csv")
wind_data = pd.read_csv("C:\\Users\\PRANAV\\Downloads\\Amprion.csv")

In [11]:
def load_data(file_path):
    data = pd.read_csv(file_path)
    # Preprocessing steps like filling missing values, normalization, etc.
    return data

In [12]:
def create_sequences(data, seq_length):
    sequences = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
    return np.array(sequences)


In [13]:
data = pd.concat([solar_data, wind_data], axis=1)

In [15]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
solar_data = pd.read_csv(r"C:\Users\PRANAV\Downloads\Plant_1_Weather_Sensor_Data.csv")
wind_data = pd.read_csv(r"C:\Users\PRANAV\Downloads\Amprion.csv")


In [16]:
print(solar_data.dtypes)
print(wind_data.dtypes)

DATE_TIME               object
PLANT_ID                 int64
SOURCE_KEY              object
AMBIENT_TEMPERATURE    float64
MODULE_TEMPERATURE     float64
IRRADIATION            float64
dtype: object
Date        object
00:00:00     int64
00:15:00     int64
00:30:00     int64
00:45:00     int64
             ...  
22:45:00     int64
23:00:00     int64
23:15:00     int64
23:30:00     int64
23:45:00     int64
Length: 97, dtype: object


In [17]:
numeric_columns_solar = solar_data.select_dtypes(include=['float64', 'int64']).columns
data_numeric_solar = solar_data[numeric_columns_solar]

numeric_columns_wind = wind_data.select_dtypes(include=['float64', 'int64']).columns
data_numeric_wind = wind_data[numeric_columns_wind]

In [18]:
scaler = MinMaxScaler()
data_scaled_solar = scaler.fit_transform(data_numeric_solar)
data_scaled_wind = scaler.fit_transform(data_numeric_wind)

data_scaled_solar = pd.DataFrame(data_scaled_solar, columns=numeric_columns_solar)
data_scaled_wind = pd.DataFrame(data_scaled_wind, columns=numeric_columns_wind)


In [19]:
if 'datetime_column_name' in solar_data.columns:
    solar_data['datetime_column_name'] = pd.to_datetime(solar_data['datetime_column_name'])
    solar_data['timestamp'] = solar_data['datetime_column_name'].astype('int64') // 10**9  

if 'datetime_column_name' in wind_data.columns:
    wind_data['datetime_column_name'] = pd.to_datetime(wind_data['datetime_column_name'])
    wind_data['timestamp'] = wind_data['datetime_column_name'].astype('int64') // 10**9  


In [25]:
SEQ_LENGTH = 60  # Example sequence length
sequences = create_sequences(data_scaled_solar, SEQ_LENGTH)
SEQ_LENGTH = 60  # Example sequence length
sequences = create_sequences(data_scaled_wind, SEQ_LENGTH)

In [26]:
train_size = int(len(sequences) * 0.8)
train_sequences = sequences[:train_size]
test_sequences = sequences[train_size:]

X_train = train_sequences[:, :-1]
y_train = train_sequences[:, -1]
X_test = test_sequences[:, :-1]
y_test = test_sequences[:, -1]

In [27]:
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(50))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mse')

In [28]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

Epoch 1/20
7/7 [==============================] - 6s 188ms/step - loss: 0.0722 - val_loss: 0.0222
Epoch 2/20
7/7 [==============================] - 0s 42ms/step - loss: 0.0574 - val_loss: 0.0192
Epoch 3/20
7/7 [==============================] - 0s 40ms/step - loss: 0.0559 - val_loss: 0.0186
Epoch 4/20
7/7 [==============================] - 0s 42ms/step - loss: 0.0515 - val_loss: 0.0171
Epoch 5/20
7/7 [==============================] - 0s 40ms/step - loss: 0.0478 - val_loss: 0.0183
Epoch 6/20
7/7 [==============================] - 0s 41ms/step - loss: 0.0454 - val_loss: 0.0156
Epoch 7/20
7/7 [==============================] - 0s 42ms/step - loss: 0.0453 - val_loss: 0.0155
Epoch 8/20
7/7 [==============================] - 0s 44ms/step - loss: 0.0413 - val_loss: 0.0148
Epoch 9/20
7/7 [==============================] - 0s 42ms/step - loss: 0.0383 - val_loss: 0.0144
Epoch 10/20
7/7 [==============================] - 0s 42ms/step - loss: 0.0364 - val_loss: 0.0158
Epoch 11/20
7/7 [===========

In [29]:
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

3/3 [==============================] - 0s 12ms/step - loss: 0.0121
Test Loss: 0.01207816880196333


In [30]:
predictions = model.predict(X_test)

3/3 [==============================] - 1s 13ms/step


In [31]:
from sklearn.ensemble import IsolationForest

# Train isolation forest on training data
iso_forest = IsolationForest(contamination=0.01)
iso_forest.fit(X_train.reshape(X_train.shape[0], -1))

IsolationForest(contamination=0.01)

In [32]:
anomalies = iso_forest.predict(X_test.reshape(X_test.shape[0], -1))
anomalies = np.where(anomalies == -1, 1, 0)

In [33]:
print(f'Number of anomalies detected: {np.sum(anomalies)}')

Number of anomalies detected: 0
